# Feature Set Generator :: ECDF

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import skew, kurtosis, linregress
from matplotlib import pyplot as plt
import multiprocessing
from multiprocessing import Pool
import os
# import math

In [2]:
# Path settings

# path_var = 1 

# if (path_var == 1):
#     filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'

# else:
#     filepath = '/home/sharan/Desktop/Option_Pricing/datasets/'


filepath = '/home/docboy/Desktop/SBIN/'


In [3]:

# clean = pd.read_csv(filepath + 'ttm_cleaned_BANKNIFTY_2014_2018_raw.csv')
# clean = clean.dropna()
# filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_BANKNIFTY_2015_2018.csv')
# ts_data = pd.read_csv(filepath + 'INDEXVALUE_BANKNIFTY_2014_2018_combined_raw.csv')


# For NIFTY50

clean = pd.read_csv(filepath + 'ttm_filter_param_0.04_SBIN_2014_2018.csv')
clean = clean.dropna()
filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_SBIN_2015_2018.csv')
ts_data = pd.read_csv(filepath + 'INDEXVALUE_SBIN_2015_2018_combined.csv')


In [4]:
#print(clean)
filt['Date'] = pd.to_datetime(filt['Date'])
filt = filt.sort_values(by='Date')
filt.reset_index(inplace = True, drop = True)
clean['Date'] = pd.to_datetime(clean['Date'])
clean['Expiry'] = pd.to_datetime(clean['Expiry'])
clean = clean.sort_values(by='Date')
clean.reset_index(inplace = True, drop = False)
ts_data['Date'] = pd.to_datetime(ts_data['Date'])  
ts_data = ts_data.sort_values(by='Date')
ts_data.reset_index(inplace = True, drop = False)

# Finding the Scale Parameter

In [5]:
filt['pct'] = filt["Close"]/filt["Strike Price"]*100

In [6]:
p = round(len(filt)*0.90) 
q = filt.sort_values(by='pct').reset_index(drop=True)
scale = q.loc[p, 'pct']/5
print('scale value is ::', scale)

scale value is :: 1.2054054054054055


# Scaling the Datasets

In [7]:
col = ['Strike Price', 'Open', 'High', 'Low', 'Close', 'Underlying Value']
clean.loc[:, col] = (clean.loc[:, col])**(1/scale)
filt.loc[:, col] = (filt.loc[:, col])**(1/scale)

In [8]:
col = ['Close', 'Open', 'Low', 'High', 'Last Price', 'Prev Close']
ts_data.loc[:, col] = (ts_data.loc[:, col])**(1/scale)

In [9]:
ts_data

,index,Symbol,Date,Prev Close,Last Price,Close,Open,High,Low
0,0,SBIN,2015-01-01,117.211020,117.881018,117.881018,117.398075,118.192379,116.852326
1,1,SBIN,2015-01-02,117.881018,118.379114,118.270192,117.990014,119.218677,117.990014
2,2,SBIN,2015-01-05,118.270192,117.507163,117.491580,118.581342,118.752403,117.288967
3,3,SBIN,2015-01-06,117.491580,113.472544,113.472544,116.633880,116.977115,113.095744
4,4,SBIN,2015-01-07,113.472544,113.943182,113.551011,113.503932,114.303735,111.979529
5,5,SBIN,2015-01-08,113.551011,115.149335,115.024141,115.071092,115.540384,114.241047
6,6,SBIN,2015-01-09,115.024141,114.444757,114.507422,115.602926,115.962412,114.131326
7,7,SBIN,2015-01-12,114.507422,115.665462,115.727991,114.804986,115.946787,113.849087
8,8,SBIN,2015-01-13,115.727991,115.149335,115.102390,116.056153,116.867926,114.804986
9,9,SBIN,2015-01-14,115.102390,115.071092,114.977187,114.758012,115.696727,114.209700


In [10]:
clean

,index,Symbol,Date,Expiry,Option Type,Strike Price,Open,High,Low,Close,LTP,Settle Price,No. of contracts,Turnover in Lacs,Open Int,Change in OI,Underlying Value,ttm
0,0,SBIN,2014-10-07,2014-10-30,CE,648.097021,25.778425,29.595349,19.164351,20.109475,38.00,37.25,3069,9577.09,162125,51875,629.696719,23
1,1,SBIN,2014-10-07,2014-10-30,CE,615.003168,63.955439,63.955439,49.749945,51.121834,114.40,114.70,28,84.70,7000,625,629.696719,23
2,2,SBIN,2014-10-07,2014-10-30,CE,626.074197,48.351541,48.351541,37.125721,38.384937,85.50,81.20,47,143.04,10750,2875,629.696719,23
3,3,SBIN,2014-10-07,2014-10-30,CE,637.105156,35.937188,39.927697,26.922353,27.805200,57.30,55.05,1981,6099.72,139000,57875,629.696719,23
4,4,SBIN,2014-10-07,2014-11-27,CE,637.105156,45.623944,45.623944,45.623944,45.623944,100.00,100.00,1,3.13,250,125,629.696719,51
5,5,SBIN,2014-10-08,2014-10-30,CE,637.105156,26.436370,38.698409,26.266903,37.461045,79.80,78.85,2839,8767.59,125000,-14000,639.581700,22
6,6,SBIN,2014-10-08,2014-10-30,CE,648.097021,18.733657,28.348841,18.710932,27.322462,55.05,53.90,3226,10075.33,149875,-12250,639.581700,22
7,7,SBIN,2014-10-08,2014-10-30,CE,659.050724,13.114150,19.997446,12.967291,19.254769,36.50,35.35,4817,15245.21,412250,9375,639.581700,22
8,8,SBIN,2014-10-08,2014-10-30,CE,626.074197,37.263870,50.862873,37.263870,50.065789,111.00,111.85,141,431.34,12875,2125,639.581700,22
9,9,SBIN,2014-10-08,2014-11-27,CE,659.050724,35.579061,35.579061,33.938037,34.560300,71.55,71.55,5,16.07,1375,250,639.581700,50


# Option Data Functions

In [11]:
def ret(data, date, mode='s'):
    '''
    This function takes in 2 primary params. The function outputs a vector of simple return data
    corressponding to previous 20 days raw ts data.
    The function can also output the log returns on changing the value of the mode param to 'log'.

    ## Modification :: The function can also output just the raw rows if mode param is set to 'e'

    # Demo func
    ret(df, '31-Jan-2019')
    ret(df, '31-Jan-2019', 'log')
    ret(df, '31-Jan-2019', 'e')

    '''
#   print(data.loc[k,:])
    if (mode == 's'): # defualt Func logic
        val = pd.DataFrame()
        val.insert(0, "Return", 0)
        temp=((data['Close']-data['Close'].shift(1))/(data['Close'].shift(1)))*100
        val['Return']=temp[1:]

    if (mode == 'log'): # Func logic if log returns is desired
        val = pd.DataFrame()
        val.insert(0, "Log Return", 0)
        temp = np.log(data['Close'])-np.log(data['Close'].shift(1))
        val['Log Return']=temp[1:]

    if (mode == 'e'): # Func logic to extract the raw data
        k = data.index[data['Date']==date][0]
        val = pd.DataFrame()
        val.insert(0, "Close", 0)
        val.insert(0, "High", 0)
        val.insert(0, "Low", 0)
        val.insert(0, "Date", 0)
        val["Close"] = data.loc[k-19:k, "Close"]
        val['High'] = data.loc[k-19:k, "High"]
        val['Low'] = data.loc[k-19:k, "Low"]
        val['Open'] = data.loc[k-19:k, "Open"]
        val['Date'] = data.loc[k-19:k, "Date"]
    val.reset_index(inplace=True, drop=True)
    return val

### Feature Generator

In [12]:
def gen(ts_data, ttm, atmDate, atmError, strike,spot, opPrice, percentage):
    
    log_return = (ret(ts_data, atmDate, mode = 'log'))
    
    log_return = log_return.loc[:,'Log Return'].tolist()
        
    log_return.sort()
    
    return [atmDate] + log_return + [ttm, atmError, strike,spot, opPrice, percentage]

In [13]:
flist=[]
for i in range(1,20):
    fname='f'+ str(i)
    flist.append(fname)
col_name = ['date'] + flist + ['ttm','atmError','strike','spot','opPrice','percentage']

In [14]:
len(filt)

6806

In [15]:
filt.reset_index(drop=True,inplace=True)
# filt
clean.reset_index(drop=True,inplace=True)

In [16]:
# clean.loc[clean["Strike Price"] == 8600.00]

In [17]:
count = 0
# percentage = []

feat = []
params=[]
for index,row in filt.iterrows():
    ttm = row['ttm']
    strike=row['Strike Price']
    spot=row['Underlying Value']
    atmDate=row['Date']
    expDate=row['Expiry']
    close = row['Close']
    opPrice = close
    opsTS = clean.loc[np.logical_and(clean['Strike Price'] == strike, clean['Expiry'] == expDate)]
    opsTS=opsTS.drop("index",1)
    opsTS.reset_index(inplace=True, drop=True)
#     print(atmDate)
    ind = opsTS.index[opsTS['Date']==atmDate]#Gives Index of that day itself
#     print(ind)
#     print(opsTS)
    ind=ind[0]
    if (ind > 20):
        window = 20
    elif (ind < 1):
        continue        
    else:
        window = ind
        
#     if((spot-strike) >= opPrice):
#         continue
    
    percentage = row['pct']/scale
    
    ts = ret(ts_data, atmDate, mode = 'e')
    
    atmError = ((spot/strike)*100)
    
    params.append((ts,ttm,atmDate,atmError,strike,spot,opPrice,percentage))
#     break
    
pool = Pool()
for res in pool.starmap(gen,params):    
    feat.append(res)
    
pool.close()


In [18]:
results=pd.DataFrame.from_records(feat, columns=col_name)
results

,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f16,f17,f18,f19,ttm,atmError,strike,spot,opPrice,percentage
0,2015-01-01,28.000000,98.442003,119.746667,117.881018,6.189223,2.333240,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,28.000000,99.736565,118.192379,117.881018,7.447875,2.962844,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,28.000000,97.183929,121.296823,117.881018,5.054226,1.799586,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01,28.000000,102.441905,115.071092,117.881018,10.796248,4.787179,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01,28.000000,101.069276,116.633880,117.881018,8.929429,3.746564,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-01-01,56.000000,101.069276,116.633880,117.881018,12.202940,5.459280,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-01-01,56.000000,99.736565,118.192379,117.881018,10.745334,4.608869,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-01-01,56.000000,98.442003,119.746667,117.881018,9.612044,3.966508,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-01-02,0.003296,55.000000,100.065837,118.192379,118.270192,11.403464,4.951242,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-01-02,0.003296,27.000000,102.780108,115.071092,118.270192,10.796248,4.787179,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
results = results.dropna()

In [20]:
print(len(results))

6067


In [21]:
# results.to_csv(filepath + 'BANKNIFTY_feature_set_ECDF_STEP_1.csv', index = False)

filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'

results.to_csv(filepath + 'SBIN_feature_set_ECDF_STEP_1_SCALED.csv', index = False)



